In [1]:
import sys
import os
import json
import xml.etree.ElementTree as ET
import glob


In [5]:

def addCatItem(name):
    global category_item_id
    category_item = dict()
    category_item['supercategory'] = 'none'
    category_item_id += 1
    category_item['id'] = category_item_id
    category_item['name'] = name
    coco['categories'].append(category_item)
    category_set[name] = category_item_id
    return category_item_id

def addImgItem(file_name, size):
    global image_id
    if file_name is None:
        raise Exception('Could not find filename tag in xml file.')
    if size['width'] is None:
        raise Exception('Could not find width tag in xml file.')
    if size['height'] is None:
        raise Exception('Could not find height tag in xml file.')
    image_id += 1
    image_item = dict()
    image_item['id'] = image_id
    image_item['file_name'] = file_name
    image_item['width'] = size['width']
    image_item['height'] = size['height']
    coco['images'].append(image_item)
    image_set.add(file_name)
    return image_id

def addAnnoItem(object_name, image_id, category_id, bbox):
    global annotation_id
    annotation_item = dict()
    annotation_item['segmentation'] = []
    seg = []
    #bbox[] is x,y,w,h
    #left_top
    seg.append(bbox[0])
    seg.append(bbox[1])
    #left_bottom
    seg.append(bbox[0])
    seg.append(bbox[1] + bbox[3])
    #right_bottom
    seg.append(bbox[0] + bbox[2])
    seg.append(bbox[1] + bbox[3])
    #right_top
    seg.append(bbox[0] + bbox[2])
    seg.append(bbox[1])

    annotation_item['segmentation'].append(seg)

    annotation_item['area'] = bbox[2] * bbox[3]
    annotation_item['iscrowd'] = 0
    annotation_item['ignore'] = 0
    annotation_item['image_id'] = image_id
    annotation_item['bbox'] = bbox
    annotation_item['category_id'] = category_id
    annotation_id += 1
    annotation_item['id'] = annotation_id
    coco['annotations'].append(annotation_item)

def parseXmlFiles(xml_path): 
    for f in os.listdir(xml_path):
        if not f.endswith('.xml'):
            continue
        
        bndbox = dict()
        size = dict()
        current_image_id = None
        current_category_id = None
        file_name = None
        size['width'] = None
        size['height'] = None
        size['depth'] = None

        xml_file = os.path.join(xml_path, f)
        print(xml_file)

        tree = ET.parse(xml_file)
        root = tree.getroot()
        if root.tag != 'annotation':
            raise Exception('pascal voc xml root element should be annotation, rather than {}'.format(root.tag))

        #elem is <folder>, <filename>, <size>, <object>
        for elem in root:
            current_parent = elem.tag
            current_sub = None
            object_name = None
            
            if elem.tag == 'folder':
                continue
            
            if elem.tag == 'filename':
                file_name = elem.text
                if file_name in category_set:
                    raise Exception('file_name duplicated')
                
            #add img item only after parse <size> tag
            elif current_image_id is None and file_name is not None and size['width'] is not None:
                if file_name not in image_set:
                    current_image_id = addImgItem(file_name, size)
                    print('add image with {} and {}'.format(file_name, size))
                else:
                    raise Exception('duplicated image: {}'.format(file_name)) 
            #subelem is <width>, <height>, <depth>, <name>, <bndbox>
            for subelem in elem:
                bndbox ['xmin'] = None
                bndbox ['xmax'] = None
                bndbox ['ymin'] = None
                bndbox ['ymax'] = None
                
                current_sub = subelem.tag
                if current_parent == 'object' and subelem.tag == 'name':
                    object_name = subelem.text
                    if object_name not in category_set:
                        current_category_id = addCatItem(object_name)
                    else:
                        current_category_id = category_set[object_name]

                elif current_parent == 'size':
                    if size[subelem.tag] is not None:
                        raise Exception('xml structure broken at size tag.')
                    size[subelem.tag] = int(subelem.text)

                #option is <xmin>, <ymin>, <xmax>, <ymax>, when subelem is <bndbox>
                for option in subelem:
                    if current_sub == 'bndbox':
                        if bndbox[option.tag] is not None:
                            raise Exception('xml structure corrupted at bndbox tag.')
                        bndbox[option.tag] = int(option.text)

                #only after parse the <object> tag
                if bndbox['xmin'] is not None:
                    if object_name is None:
                        raise Exception('xml structure broken at bndbox tag')
                    if current_image_id is None:
                        raise Exception('xml structure broken at bndbox tag')
                    if current_category_id is None:
                        raise Exception('xml structure broken at bndbox tag')
                    bbox = []
                    #x
                    bbox.append(bndbox['xmin'])
                    #y
                    bbox.append(bndbox['ymin'])
                    #w
                    bbox.append(bndbox['xmax'] - bndbox['xmin'])
                    #h
                    bbox.append(bndbox['ymax'] - bndbox['ymin'])
                    print('add annotation with {},{},{},{}'.format(object_name, current_image_id, current_category_id, bbox))
                    addAnnoItem(object_name, current_image_id, current_category_id, bbox )


In [4]:
import xml.etree.ElementTree as ET
import os
import json

coco = dict()
coco['images'] = []
coco['type'] = 'instances'
coco['annotations'] = []
coco['categories'] = []

category_set = dict()
image_set = set()

category_item_id = 0
image_id = 20180000000
annotation_id = 0

xml_path = 'xml/train/'
json_file = 'train.json'
parseXmlFiles(xml_path)
json.dump(coco, open(json_file, 'w'))

xml/train/1.xml
add image with 1.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180000001,1,[56, 96, 790, 522]
add annotation with table,20180000001,1,[49, 100, 792, 517]
xml/train/10.xml
add image with 10.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180000002,1,[52, 312, 792, 204]
add annotation with table,20180000002,1,[50, 532, 795, 167]
add annotation with table,20180000002,1,[55, 129, 785, 164]
xml/train/100.xml
add image with 100.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000003,1,[66, 488, 693, 261]
add annotation with table,20180000003,1,[59, 793, 700, 288]
add annotation with table,20180000003,1,[62, 320, 698, 116]
add annotation with table,20180000003,1,[66, 131, 692, 142]
xml/train/1000.xml
add image with 1000.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000004,1,[29, 144, 716, 645]
xml/train/1001.xml
add image with 1001.png and {'widt

add annotation with table,20180000264,1,[60, 471, 757, 179]
xml/train/1236.xml
add image with 1236.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000265,1,[300, 178, 456, 77]
xml/train/1237.xml
add image with 1237.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180000266,1,[52, 739, 779, 316]
add annotation with table,20180000266,1,[59, 377, 773, 83]
xml/train/1238.xml
add image with 1238.png and {'width': 827, 'height': 1063, 'depth': 3}
add annotation with table,20180000267,1,[76, 283, 677, 216]
xml/train/1239.xml
add image with 1239.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000268,1,[45, 731, 720, 262]
add annotation with table,20180000268,1,[44, 430, 722, 263]
xml/train/124.xml
add image with 124.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180000269,1,[58, 463, 773, 271]
add annotation with table,20180000269,1,[58, 983, 771, 118]
add annotatio

add image with 1448.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180000500,1,[51, 169, 788, 1039]
xml/train/1449.xml
add image with 1449.png and {'width': 867, 'height': 1103, 'depth': 3}
add annotation with table,20180000501,1,[93, 769, 713, 96]
add annotation with table,20180000501,1,[95, 479, 712, 178]
xml/train/145.xml
add image with 145.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180000502,1,[72, 567, 725, 112]
add annotation with table,20180000502,1,[66, 916, 732, 101]
add annotation with table,20180000502,1,[87, 149, 711, 78]
add annotation with table,20180000502,1,[91, 435, 707, 116]
add annotation with table,20180000502,1,[71, 747, 728, 101]
add annotation with table,20180000502,1,[97, 336, 700, 61]
add annotation with table,20180000502,1,[64, 1031, 732, 81]
xml/train/1450.xml
add image with 1450.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000503,1,[56, 221, 711, 131]
add

add annotation with table,20180000718,1,[49, 405, 723, 86]
add annotation with table,20180000718,1,[49, 579, 725, 130]
add annotation with table,20180000718,1,[52, 778, 722, 91]
xml/train/1645.xml
add image with 1645.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000719,1,[70, 195, 688, 224]
xml/train/1646.xml
add image with 1646.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180000720,1,[53, 199, 786, 268]
xml/train/1647.xml
add image with 1647.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000721,1,[51, 177, 716, 300]
add annotation with table,20180000721,1,[52, 500, 716, 303]
add annotation with table,20180000721,1,[54, 830, 715, 117]
xml/train/1648.xml
add image with 1648.png and {'width': 1786, 'height': 1263, 'depth': 3}
add annotation with table,20180000722,1,[73, 274, 737, 94]
add annotation with table,20180000722,1,[977, 929, 742, 67]
add annotation with table,20180000722,1,[6

add image with 1844.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000940,1,[95, 560, 625, 111]
add annotation with table,20180000940,1,[96, 912, 625, 149]
add annotation with table,20180000940,1,[96, 178, 624, 84]
add annotation with table,20180000940,1,[96, 755, 624, 145]
add annotation with table,20180000940,1,[96, 267, 623, 58]
xml/train/1845.xml
add image with 1845.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000941,1,[48, 485, 704, 97]
add annotation with table,20180000941,1,[51, 639, 702, 220]
xml/train/1846.xml
add image with 1846.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180000942,1,[29, 161, 723, 351]
add annotation with table,20180000942,1,[32, 550, 718, 336]
add annotation with table,20180000942,1,[34, 552, 712, 334]
add annotation with table,20180000942,1,[35, 159, 717, 354]
xml/train/1847.xml
add image with 1847.png and {'width': 847, 'height': 1103, 'depth': 3}
ad

...

add image with 4242.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180003605,1,[122, 920, 649, 109]
xml/train/4243.xml
add image with 4243.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180003606,1,[35, 699, 783, 159]
add annotation with table,20180003606,1,[40, 904, 777, 210]
add annotation with table,20180003606,1,[36, 458, 780, 193]
add annotation with table,20180003606,1,[37, 252, 779, 158]
xml/train/4244.xml
add image with 4244.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180003607,1,[57, 653, 768, 157]
xml/train/4245.xml
add image with 4245.png and {'width': 945, 'height': 1327, 'depth': 3}
add annotation with table,20180003608,1,[170, 314, 685, 561]
xml/train/4246.xml
add image with 4246.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180003609,1,[49, 234, 724, 281]
add annotation with table,20180003609,1,[51, 615, 723, 173]
xml/train/4247.xml
add imag

xml/train/4485.xml
add image with 4485.png and {'width': 1786, 'height': 1263, 'depth': 3}
add annotation with table,20180003874,1,[1209, 294, 515, 69]
xml/train/4486.xml
add image with 4486.png and {'width': 1786, 'height': 1263, 'depth': 3}
add annotation with table,20180003875,1,[953, 196, 771, 422]
add annotation with table,20180003875,1,[57, 203, 774, 219]
xml/train/4487.xml
add image with 4487.png and {'width': 1786, 'height': 1263, 'depth': 3}
add annotation with table,20180003876,1,[977, 908, 742, 144]
add annotation with table,20180003876,1,[970, 474, 753, 99]
xml/train/4488.xml
add image with 4488.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180003877,1,[49, 399, 344, 189]
xml/train/4489.xml
add image with 4489.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180003878,1,[59, 259, 716, 252]
xml/train/449.xml
add image with 449.png and {'width': 827, 'height': 1063, 'depth': 3}
add annotation with table,2018000387

add annotation with table,20180004104,1,[72, 680, 707, 84]
xml/train/530.xml
add image with 530.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180004105,1,[55, 286, 765, 50]
add annotation with table,20180004105,1,[59, 861, 757, 91]
add annotation with table,20180004105,1,[57, 408, 760, 412]
xml/train/531.xml
add image with 531.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180004106,1,[69, 267, 688, 223]
xml/train/532.xml
add image with 532.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180004107,1,[56, 717, 713, 257]
add annotation with table,20180004107,1,[56, 180, 712, 336]
xml/train/533.xml
add image with 533.png and {'width': 945, 'height': 1327, 'depth': 3}
add annotation with table,20180004108,1,[169, 312, 689, 594]
xml/train/534.xml
add image with 534.png and {'width': 847, 'height': 1063, 'depth': 3}
add annotation with table,20180004109,1,[57, 558, 751, 62]
add annotation with tab

add annotation with table,20180004324,1,[59, 752, 714, 309]
add annotation with table,20180004324,1,[57, 244, 718, 249]
xml/train/729.xml
add image with 729.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180004325,1,[60, 402, 772, 235]
add annotation with table,20180004325,1,[58, 678, 775, 264]
xml/train/73.xml
add image with 73.png and {'width': 893, 'height': 1263, 'depth': 3}
add annotation with table,20180004326,1,[57, 444, 774, 68]
add annotation with table,20180004326,1,[62, 713, 769, 196]
add annotation with table,20180004326,1,[57, 201, 772, 118]
add annotation with table,20180004326,1,[59, 581, 774, 90]
add annotation with table,20180004326,1,[56, 445, 778, 67]
add annotation with table,20180004326,1,[49, 712, 781, 197]
add annotation with table,20180004326,1,[57, 1022, 773, 87]
add annotation with table,20180004326,1,[53, 1019, 779, 91]
add annotation with table,20180004326,1,[43, 199, 790, 120]
add annotation with table,20180004326,1,[60, 581,

add image with 919.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180004536,1,[76, 843, 704, 72]
add annotation with table,20180004536,1,[77, 214, 703, 151]
add annotation with table,20180004536,1,[77, 1031, 702, 71]
add annotation with table,20180004536,1,[77, 510, 703, 279]
add annotation with table,20180004536,1,[77, 379, 703, 72]
xml/train/92.xml
add image with 92.png and {'width': 827, 'height': 1170, 'depth': 3}
add annotation with table,20180004537,1,[58, 733, 711, 78]
add annotation with table,20180004537,1,[56, 253, 718, 194]
add annotation with table,20180004537,1,[57, 863, 712, 77]
add annotation with table,20180004537,1,[56, 952, 714, 77]
add annotation with table,20180004537,1,[56, 472, 712, 223]
xml/train/920.xml
add image with 920.png and {'width': 847, 'height': 1063, 'depth': 3}
add annotation with table,20180004538,1,[37, 780, 750, 123]
add annotation with table,20180004538,1,[37, 507, 751, 124]
xml/train/921.xml
add image with 921.png 